# GRACE Foundation Model Distillation

## Overview

This notebook demonstrates **knowledge distillation** for a GRACE foundation model. The process involves:

1. **Training Data Generation**: Create diverse structures from Materials Project (MP) data
2. **Model Evaluation**: Use a finetuned foundation model to predict energies, forces, and stresses
3. **Data Augmentation**: Apply structural deformations and perturbations to expand the dataset
4. **Dataset Creation**: Compile all predictions into a distilled dataset for training a smaller student model

### What is Distillation?

Distillation transfers knowledge from a large pretrained model (teacher) to a smaller model (student). Instead of computing expensive DFT calculations, we use the teacher model's predictions as training targets. This significantly reduces computational cost while maintaining accuracy.

### Workflow

```
Materials Project Structures + DFT calculations
         ↓
    Finetune founational GRACE-2L-OMAT-medium-ft-E
         ↓
     GRACE-2L-ft-AlLi
         ↓
   Structural Deformations (Murnaghan, Random Strains)
         ↓
   Model Predictions (E, F, σ)
         ↓
   Distilled Training Dataset
```

In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## Step 1: Import Libraries

First, we import necessary libraries for data handling, ASE structures, and GRACE model evaluation.

In [2]:
import pandas as pd

In [3]:
from amstools.sources import fetch_structures, fetch_mp_reference_df

In [4]:
from amstools.qatoms import  QAtoms

In [5]:
from amstools.properties import MurnaghanCalculator, RandomDeformationCalculator, StepwiseOptimizer

In [6]:
from tensorpotential.calculator import grace_fm, TPCalculator

[tensorpotential] Info: Environment variable TF_USE_LEGACY_KERAS is automatically set to '1'.


2026-02-26 20:17:47.048637: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-26 20:17:47.060588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1772133467.074729 1326266 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1772133467.078917 1326266 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1772133467.089776 1326266 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [7]:
#MP_API_KEY="YOUR_API_KEY"

## Step 2: Load Reference Structures from Materials Project

We fetch reference structures for Al and Li from the Materials Project database. These structures serve as the basis for our distilled dataset. We limit to structures with ≤32 atoms for computational efficiency.

**Parameters:**
- `elements`: ['Al','Li'] - Elements to include
- `max_atoms`: 32 - Maximum system size
- `cache_dir`: Local cache for downloaded structures

In [8]:
ref_df=fetch_mp_reference_df(['Al','Li'], 
                             # mp_api_key=MP_API_KEY,
                             max_atoms=32,
                             cache_dir='../../1-AlLi-GRACE-2LAYER/0-data/AlLi_Materials_Project/')

### Loaded Structures

Below we can see the reference dataframe containing all downloaded structures:

In [9]:
ref_df

,name,ase_atoms,e_per_atom,e_formation_per_atom,e_chull_dist_per_atom
0,Al__mp-1183144,"(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom('A...",NaN,0.010481,0.010481
1,Al__mp-2647008,"(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom('A...",NaN,0.032078,0.032078
2,Al__mp-134,"(Atom('Al', [0.0, 0.0, 0.0], index=0))",NaN,0.000000,0.000000
3,Li__mp-567337,"(Atom('Li', [-0.9218888897176146, 2.8025429460...",NaN,0.008819,0.008819
4,Li__mp-10173,"(Atom('Li', [-1.1807313597921131e-06, 1.740594...",NaN,0.005988,0.005988
5,Li__mp-51,"(Atom('Li', [0.0, 0.0, 0.0], index=0))",NaN,0.001672,0.001672
6,Li__mp-976411,"(Atom('Li', [0.0, 0.0, 1.0216207669999998e-05]...",NaN,0.005889,0.005889
7,Li__mp-1103107,"(Atom('Li', [1.379696372079909, 3.982485883382...",NaN,0.055379,0.055379
8,Li__mp-1063005,"(Atom('Li', [0.0, 0.0, 0.0], index=0), Atom('L...",NaN,0.015671,0.015671
9,Li__mp-1018134,"(Atom('Li', [4.618009551014942, 2.666207852974...",NaN,0.000000,0.000000


## Step 3: Load the Finetuned Teacher Model

We load the finetuned GRACE foundation model that has been trained on Al-Li data. This model serves as the **teacher** that will provide predictions for the distilled dataset.

The model is loaded as an ASE calculator, allowing us to compute:
- Potential energies (E)
- Atomic forces (F)
- Stress tensors (σ)

In [10]:
calc=TPCalculator("../3a-finetuning/seed/1/final_model/")

2026-02-26 20:17:50.209660: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1772133470.209878 1326266 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6245 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:b3:00.0, compute capability: 8.6


## Step 4: Prepare Structure Dictionary

Convert the Materials Project structures into a dictionary for easy access. We also add some random structures to increase dataset diversity.

In [11]:
struct_dict=ref_df.set_index("name")['ase_atoms'].to_dict()

### Adding Random Structures for Dataset Diversity

We create some random defective structures by randomly replacing atoms with Li in a 2×2×2 supercell of bulk Al. This adds diversity to the training set and helps the model generalize better.

In [12]:
for i in range(20):
    small_structure = (QAtoms.bulk("Al",cubic=True)*(2,2,2)).sample(frac=0.01+0.01*i, random_state=i).set(element='Li')
    print(f"random_struct_{i} has {len(small_structure)} atoms and {small_structure.get_chemical_formula()}.")
    struct_dict[f'random_struct_{i}'] = small_structure

random_struct_0 has 32 atoms and Al31Li.
random_struct_1 has 32 atoms and Al31Li.
random_struct_2 has 32 atoms and Al31Li.
random_struct_3 has 32 atoms and Al30Li2.
random_struct_4 has 32 atoms and Al30Li2.
random_struct_5 has 32 atoms and Al30Li2.
random_struct_6 has 32 atoms and Al29Li3.
random_struct_7 has 32 atoms and Al29Li3.
random_struct_8 has 32 atoms and Al29Li3.
random_struct_9 has 32 atoms and Al28Li4.
random_struct_10 has 32 atoms and Al28Li4.
random_struct_11 has 32 atoms and Al28Li4.
random_struct_12 has 32 atoms and Al27Li5.
random_struct_13 has 32 atoms and Al27Li5.
random_struct_14 has 32 atoms and Al27Li5.
random_struct_15 has 32 atoms and Al26Li6.
random_struct_16 has 32 atoms and Al26Li6.
random_struct_17 has 32 atoms and Al26Li6.
random_struct_18 has 32 atoms and Al25Li7.
random_struct_19 has 32 atoms and Al25Li7.


## Step 5: Generate Structural Deformations & Collect Model Predictions

This is the core of the distillation process. For each structure, we apply multiple deformations:

### Pipeline Components:

1. **StepwiseOptimizer**: Relaxes atomic positions using the GRACE model (keeps cell fixed)
2. **MurnaghanCalculator**: Applies isotropic volume deformations to find equation of state
3. **RandomDeformationCalculator**: Applies random atomic displacements and cell strains

### Parameters:
- **supercell_size**: (1,1,2) - Creates slab-like geometry
- **nsample**: 3 - Number of random deformation samples
- **random_atom_displacement**: 0.5 Å - Magnitude of atomic perturbations
- **random_cell_strain**: 0.15 - Magnitude of cell strains
- **volume_range**: 0.25 - Volume deformation range (±12.5%)
- **num_volume_deformations**: 5 - Number of different volumes to sample

For each deformation, the model predicts E, F, σ which become training data.

In [53]:
from amstools import  properties as p

p.DefectFormationCalculator

amstools.properties.defectformation.DefectFormationCalculator

In [56]:
pipelines_dict={}

In [57]:
for i,(struct_name, at) in enumerate(struct_dict.items()):
    # Only process each structure once
    if struct_name not in pipelines_dict:
        at.calc = calc  # Attach the finetuned GRACE model as calculator
        
        # Build the deformation pipeline
        pipe = (
            # Option 1: Use nearest neighbor distance scanning
            # NearestNeighboursExpansionCalculator(nn_distance_range=(2,7), nn_distance_step=1) 
            
            # Step 1: Atomic relaxation (fixed cell)
            StepwiseOptimizer() 
            
            # Step 2: Equation of state (isotropic volume changes)
            + MurnaghanCalculator(optimize_deformed_structure=False) 
            
            # Step 3: Random deformations (supercell + atomic displacements + cell strains)
            + RandomDeformationCalculator(
                supercell_size=(2,2,2) if len(at)<32 else (2,2,1),           # Create supercell geometry
                nsample=3,                        # 3 random samples 
                random_atom_displacement=0.5,     # Å - random atomic perturbations
                random_cell_strain=0.15,          # Random cell deformations
                volume_range=0.125,                # ±12.5% volume change
                num_volume_deformations=5,        # Sample 5 different volumes
                seed = i
            )
        )

        if not struct_name.startswith("random"):
            pipe +=  DefectFormationCalculator(type='vacancy')
        # break
        # Execute the pipeline on this structure
        # This generates all deformed configurations automatically
        pipe.run(init_structure=at, engine=calc, verbose=True)
        pipelines_dict[struct_name] = pipe
        

2026-02-26 20:30:16,142 - root - INFO - =======================
2026-02-26 20:30:16,144 - root - INFO - Step: optimization
Processing: optimized_structure (4 atom(s)): 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.01it/s]
2026-02-26 20:30:16,263 - root - INFO - Step status: finished
2026-02-26 20:30:16,264 - root - INFO - =======================
2026-02-26 20:30:16,265 - root - INFO - =======================
2026-02-26 20:30:16,266 - root - INFO - Step: murnaghan
Processing: strain_1_1 (4 atom(s)): 100%|█████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 74.34it/s]
2026-02-26 20:30:16,422 - root - INFO - Job is done
2026-02-26 20:30:16,423 - root - INFO - Step status: finished
2026-02-26 20:30:16,424 - root - INFO - =======================
2026-02-26 20:30:16,426 - root - INFO - =======================
2026-02-26 20:30:16,426 - root - INFO - Step: randomdeformation
Processing: rnd_2__vol_

ase.spacegroup.symmetrize: prec 0.01 got symmetry group number 187 , international (Hermann-Mauguin) P-6m2 , Hall  P -6 2
ase.spacegroup.symmetrize: prec 0.01 got symmetry group number 187 , international (Hermann-Mauguin) P-6m2 , Hall  P -6 2
ase.spacegroup.symmetrize: prec 0.0001 got symmetry group number 187 , international (Hermann-Mauguin) P-6m2 , Hall  P -6 2
ase.spacegroup.symmetrize: prec 0.01 got symmetry group number 187 , international (Hermann-Mauguin) P-6m2 , Hall  P -6 2
      Step     Time          Energy          fmax
BFGS:    0 20:30:17     -473.911438        0.213997
BFGS:    1 20:30:17     -473.918777        0.197401
BFGS:    2 20:30:17     -473.959769        0.057883
BFGS:    3 20:30:17     -473.960421        0.050981
BFGS:    4 20:30:18     -473.964082        0.020034
BFGS:    5 20:30:18     -473.964676        0.017421
BFGS:    6 20:30:18     -473.964859        0.018041
BFGS:    7 20:30:18     -473.964995        0.015564
BFGS:    8 20:30:18     -473.965303        0

Processing: supercell_defect_wyck_c_total (127 atom(s)):  29%|████████████████▌                                         | 2/7 [00:02<00:00, 11.11it/s]

BFGS:   14 20:30:19     -473.966072        0.005936
BFGS:   15 20:30:19     -473.966107        0.004603
ase.spacegroup.symmetrize: prec 0.01 got symmetry group number 187 , international (Hermann-Mauguin) P-6m2 , Hall  P -6 2
ase.spacegroup.symmetrize: prec 0.01 got symmetry group number 187 , international (Hermann-Mauguin) P-6m2 , Hall  P -6 2
ase.spacegroup.symmetrize: prec 0.0001 got symmetry group number 187 , international (Hermann-Mauguin) P-6m2 , Hall  P -6 2
ase.spacegroup.symmetrize: prec 0.01 got symmetry group number 187 , international (Hermann-Mauguin) P-6m2 , Hall  P -6 2


Processing: supercell_defect_wyck_a_atomic (127 atom(s)):  57%|████████████████████████████████▌                        | 4/7 [00:03<00:02,  1.03it/s]

...

ase.spacegroup.symmetrize: prec 0.01 got symmetry group number 221 , international (Hermann-Mauguin) Pm-3m , Hall  -P 4 2 3
ase.spacegroup.symmetrize: prec 0.01 got symmetry group number 221 , international (Hermann-Mauguin) Pm-3m , Hall  -P 4 2 3
ase.spacegroup.symmetrize: prec 0.0001 got symmetry group number 221 , international (Hermann-Mauguin) Pm-3m , Hall  -P 4 2 3
ase.spacegroup.symmetrize: prec 0.01 got symmetry group number 221 , international (Hermann-Mauguin) Pm-3m , Hall  -P 4 2 3
      Step     Time          Energy          fmax
BFGS:    0 20:31:10     -363.321005        0.205504
BFGS:    1 20:31:11     -363.328123        0.191491
BFGS:    2 20:31:11     -363.373017        0.033987
BFGS:    3 20:31:11     -363.373531        0.030522
BFGS:    4 20:31:11     -363.375890        0.018643
BFGS:    5 20:31:11     -363.376943        0.014461
BFGS:    6 20:31:11     -363.377367        0.011571
BFGS:    7 20:31:11     -363.377513        0.010464
BFGS:    8 20:31:11     -363.377736 

Processing: supercell_defect_wyck_a_total (107 atom(s)):  86%|█████████████████████████████████████████████████▋        | 6/7 [00:06<00:01,  1.17s/it]

BFGS:   10 20:31:12     -363.378103        0.005417
BFGS:   11 20:31:12     -363.378147        0.004753
ase.spacegroup.symmetrize: prec 0.01 got symmetry group number 221 , international (Hermann-Mauguin) Pm-3m , Hall  -P 4 2 3
ase.spacegroup.symmetrize: prec 0.01 got symmetry group number 221 , international (Hermann-Mauguin) Pm-3m , Hall  -P 4 2 3
ase.spacegroup.symmetrize: prec 0.0001 got symmetry group number 221 , international (Hermann-Mauguin) Pm-3m , Hall  -P 4 2 3
ase.spacegroup.symmetrize: prec 0.01 got symmetry group number 221 , international (Hermann-Mauguin) Pm-3m , Hall  -P 4 2 3


Processing: supercell_defect_wyck_a_total (107 atom(s)): 100%|██████████████████████████████████████████████████████████| 7/7 [00:07<00:00,  1.05s/it]
2026-02-26 20:31:13,121 - root - ERROR - Error has been occurred: Vacancy formation energy calculation is only valid for unary systems. Found 2 species: ['Al', 'Li']. Please compute manually with proper chemical potentials.
2026-02-26 20:31:13,122 - root - INFO - Error has been occurred: Vacancy formation energy calculation is only valid for unary systems. Found 2 species: ['Al', 'Li']. Please compute manually with proper chemical potentials.
Traceback (most recent call last):
  File "/fscratch/conda/tp2/lib/python3.10/site-packages/amstools/pipeline/pipeline.py", line 238, in iterate
    step.run(verbose=verbose)
  File "/fscratch/conda/tp2/lib/python3.10/site-packages/amstools/pipeline/pipelinestep.py", line 397, in run
    raise e
  File "/fscratch/conda/tp2/lib/python3.10/site-packages/amstools/pipeline/pipelinestep.py", line 378, in

NotImplementedError: Vacancy formation energy calculation is only valid for unary systems. Found 2 species: ['Al', 'Li']. Please compute manually with proper chemical potentials.

In [ ]:
# pipe

# pipe.run(at,calc,verbose=True)

## Step 6: Extract Predictions and Create DataFrame

Now we collect the energies, forces, and stresses from all deformed structures. Each entry in the resulting dataframe will contain:
- **Energy** (eV): Total potential energy
- **Forces** (eV/Å): Atomic forces
- **Stress** (eV/A$^3$): Stress tensor
- **ASE Atoms object**: The atomic structure

This becomes our **distilled training dataset**.

In [ ]:
from tqdm import tqdm

In [ ]:
results_data = []

# Iterate through all structures and their deformations
for struct_name, pipe in tqdm(pipelines_dict.items()):
    # Each pipeline step generates output structures
    for step_name, step in pipe.steps.items():
        # Each step can have multiple deformed structures
        for def_name, at in step.output_structures_dict.items():
            # Create a unique identifier for this structure
            name = f"{struct_name}/{step_name}/{def_name}"
            
            # Attach calculator and evaluate
            at.calc = calc 
            energy = at.get_potential_energy()
            forces = at.get_forces()
            stress = at.get_stress()
            
            # Store the data
            results_data.append({
                "name": name, 
                "ase_atoms": at, 
                "energy": energy,
                "forces": forces, 
                "stress": stress
            })
            
            # Detach calculator (free memory)
            at.calc = None


In [ ]:
df_dist=pd.DataFrame(results_data)

### Computing Per-Atom Properties

Calculate useful statistics for each entry:

In [ ]:
df_dist['nat']=df_dist['ase_atoms'].map(len)

In [ ]:
df_dist['energy_per_atom']=df_dist['energy']/df_dist['nat']

In [ ]:
df_dist

### System Size Distribution

Distribution of the number of atoms in each structure:

In [ ]:
df_dist['nat'].hist(bins=100);

## Step 7: Analyze Dataset Distribution

Let's visualize the properties of the distilled dataset to understand its characteristics:

### Energy Distribution

Energy per atom across all structures. Different peaks correspond to different phases and compositions:

In [ ]:
df_dist['energy_per_atom'].hist(bins=100);
plt.yscale('log')

### Force Distribution

Distribution of forces (log scale). Most forces are small with occasional large forces from deformed structures:

In [ ]:
plt.hist(np.vstack(df_dist['forces']).flatten(), bins=100);
plt.yscale('log')

### Stress Distribution

Stress components (in GPa) show the range of pressures sampled. Log scale reveals even small stresses are well-sampled:

In [ ]:
plt.hist(np.vstack(df_dist['stress']).flatten()*160.2, bins=100);
plt.yscale('log')

## Step 8: Save the Distilled Dataset

Export the complete distilled dataset as a compressed pickle file. This dataset can now be used to train a smaller, faster student model using the gracemaker training pipeline.

**Output file**: `distilled_AlLi_dataset.pkl.gz` - Contains all structures with their E, F, σ predictions from the teacher model.

In [ ]:
df_dist.to_pickle("distilled_AlLi_dataset.pkl.gz")

## Summary

We have successfully created a **distilled dataset** for training a student GRACE model:

### Key Achievements:

1. ✅ **Loaded Materials Project structures** - Gathered reference data for Al and Li
2. ✅ **Applied structural deformations** - Generated diverse configurations through:
   - Atomic relaxation (StepwiseOptimizer)
   - Equation of state sampling (Murnaghan deformations)
   - Random perturbations (displacements + strains)
3. ✅ **Collected predictions** - Used finetuned GRACE foundation model to evaluate all structures
4. ✅ **Created training dataset** - Compiled energies, forces, and stresses for ~1000+ configurations
5. ✅ **Saved for further use** - Output: `distilled_AlLi_dataset.pkl.gz`

### Next Steps:

The distilled dataset can now be used to:
- Train a lightweight student model (GRACE/FS)
- Deploy on CPU-only systems or use KOKKOS acceleration
- Fine-tune with additional DFT data for hybrid training

### Computational Advantages:

- **No expensive DFT calculations** needed (teacher model does predictions)
- **Diverse structure sampling** ensures good coverage of configuration space
- **Transferable knowledge** from pretrained foundation model
- **Faster inference** with smaller student models compared to foundation model